In [18]:
import pandas as pd
import numpy as np
import os.path as op


bids_folder = '/Users/mrenke/data/ds-dnumrisk'
target_folder = op.join(bids_folder, 'derivatives','phenotype')
target_folder

'/Users/mrenke/data/ds-dnumrisk/derivatives/phenotype'

In [10]:
tab_1 = pd.read_excel(op.join(bids_folder,'add_tables','add_measure_1.xlsx'), sheet_name='Corsi-Block-Tapping')
tab_1 = tab_1.dropna(how='all').dropna(how='all',axis=1).drop(2) # .loc[range(1,33),:]
tab_1 = tab_1.rename(mapper={1:'subject'}).rename(columns={'Unnamed: 1':'item_level','Unnamed: 2':'sequence' })
tab_1['item_level'] = tab_1.apply(lambda x: len(x['sequence'].split()) if isinstance(x['sequence'], str) else 0, axis=1)
#tab_1.head()

In [11]:
tab_2 = pd.read_excel(op.join(bids_folder,'add_tables','add_measure_2.xlsx'), sheet_name='Corsi-Block-Tapping')
tab_2 = tab_2.dropna(how='all').dropna(how='all',axis=1)
tab_2 = tab_2.rename(mapper={2:'subject'}).rename(columns={'Unnamed: 1':'item_level','Unnamed: 2':'sequence' })
tab_2['item_level'] = tab_2.apply(lambda x: len(x['sequence'].split()) if isinstance(x['sequence'], str) else 0, axis=1)
#tab_2.head()

In [12]:
tab_3 = pd.read_excel(op.join(bids_folder,'add_tables','add_measure_3.xlsx'), sheet_name='Corsi-Block-Tapping')
tab_3 = tab_3.dropna(how='all').dropna(how='all',axis=1)
tab_3 = tab_3.rename(mapper={2:'subject'}).rename(columns={'Unnamed: 1':'item_level','Unnamed: 2':'sequence' })
tab_3['item_level'] = tab_3.apply(lambda x: len(x['sequence'].split()) if isinstance(x['sequence'], str) else 0, axis=1)
#tab_3.head()

In [20]:
def cleanup_corsi_table(tab):
    # needs first two columns item_level & sequence, subjects in first row (starting at 3rd column)

    new_columns = tab.loc['subject'].values # put subjects into columns
    column_mapping = {old: new for old, new in zip(tab.columns[2:], new_columns[2:])}

    tab = tab.rename(columns=column_mapping).drop('subject')
    tab = tab.set_index(['item_level','sequence'])

    t = tab.T.fillna(0)

    #overall score
    overall_score =  t.sum(axis=1)

    # Erfassungsspanne
    highest_non_zero_item_level = t.apply(lambda row: max((item_level[0] for item_level, value in row.items() if value != 0), default=0), axis=1)

    # basisscore
    first_zero_indices = (tab == 0).idxmax().values
    highest_all_one_item_level = [item[0]-1 for item in first_zero_indices]
    highest_all_one_item_level = pd.DataFrame(highest_all_one_item_level,index = highest_non_zero_item_level.index)

    # names columns
    highest_all_one_item_level.columns = ['basisscore']
    overall_score = overall_score.to_frame(name='overall_score')
    highest_non_zero_item_level = highest_non_zero_item_level.to_frame(name='erfassungsspanne')

    df_comb = highest_all_one_item_level.join(overall_score).join(highest_non_zero_item_level)
    df_comb.index = df_comb.index.astype(int)
    df_comb.index.name = 'subject'

    return df_comb

In [21]:
tab_1_clean = cleanup_corsi_table(tab_1)
tab_2_clean = cleanup_corsi_table(tab_2)
tab_3_clean = cleanup_corsi_table(tab_3)

tab_comb = pd.concat([tab_1_clean, tab_2_clean, tab_3_clean], axis=0)
tab_comb.to_csv(op.join(target_folder, 'visio-spatial-WM_CBTtask-params.csv'))

/var/folders/3k/8g0xv78x051fznwyh_m5xcn8f91w3q/T/ipykernel_31828/1377248439.py:13: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  overall_score =  t.sum(axis=1)
/var/folders/3k/8g0xv78x051fznwyh_m5xcn8f91w3q/T/ipykernel_31828/1377248439.py:13: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  overall_score =  t.sum(axis=1)
